In [1]:
import jax

from quantum_transformers.datasets import get_imdb_dataloaders
from quantum_transformers.training import train_and_evaluate
from quantum_transformers.transformers import Transformer

data_dir = '/global/cfs/cdirs/m4392/salcc/data'

2023-08-26 08:09:18.298717: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-08-26 08:09:18.298744: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-08-26 08:09:18.298764: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-26 08:09:19.625187: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Please first ``pip install -U cirq`` to enable related functionality in translation module


In [2]:
for d in jax.devices():
    print(d, d.device_kind)

gpu:0 NVIDIA A100-SXM4-40GB


In [3]:
(imdb_train_dataloader, imdb_valid_dataloader), vocab, tokenizer = get_imdb_dataloaders(batch_size=32, data_dir=data_dir, max_vocab_size=20_000, max_seq_len=512)
print(f"Vocabulary size: {len(vocab)}")
first_batch = next(iter(imdb_train_dataloader))
print(first_batch[0][0])
print(' '.join(map(bytes.decode, tokenizer.detokenize(first_batch[0])[0].numpy().tolist())))

Vocabulary size: 19169
[   50   524    10    61   356   104   103   109  6014   128  4286    15
   101   159   121    95   364    97    42  2052    16   943  4244   281
    16   598    17   251    10    61   123   118   191  2008    97   767
   413   395   132  1795    34     4    34   251    10    61   123   213
   104   102    95  1302    16  9488    95   660   102    42   521   281
    16    96    18   132  6748    16   109   233   122   313   119    42
   207   317  2319    34   132   211    50   166    95  1396   448    15
   105   101   521    98   119    42  1021    96  3437   154    97   216
    96  1722    15  2921   388   102    95   286  7680    97   201    15
   132    42  1848    98  5098   114    98  1035   108   959    15   132
   232   129   104    34    34   101   242    10    61   156   645    15
   441   185   101   773   136   448    96    98   162   103   109  1301
   113   121  7067    17    31   100    18    33    31   100    18    33
   175    97   121    29    

In [4]:
model = Transformer(num_tokens=len(vocab), max_seq_len=512, num_classes=2, hidden_size=64, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=32)
train_and_evaluate(model, imdb_train_dataloader, imdb_valid_dataloader, num_classes=2, learning_rate=1e-3, num_epochs=30)

Epoch  30/30: 100%|██████████| 781/781 [00:07<00:00, 100.20batch/s, Loss = 1.6081, AUC = 87.66%]

TOTAL TIME = 249.81s
BEST AUC = 94.65% AT EPOCH 1


In [5]:
model = Transformer(num_tokens=len(vocab), max_seq_len=512, num_classes=2, hidden_size=8, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=4)
train_and_evaluate(model, imdb_train_dataloader, imdb_valid_dataloader, num_classes=2, learning_rate=1e-3, num_epochs=30)

Epoch  30/30: 100%|██████████| 781/781 [00:06<00:00, 112.65batch/s, Loss = 1.1859, AUC = 88.11%]

TOTAL TIME = 225.04s
BEST AUC = 92.14% AT EPOCH 4
